# OpenAI Simple Chat Model

In [2]:
# ! pip3 install openai

In [1]:
from agentquest.benchmarks.mastermind import MasterMindDriver, MasterMindUtils
from dotenv import load_dotenv

load_dotenv()
game = MasterMindUtils.load_data(data_path="__default__", category="4 digits")[5]
game

'9439'

Initialize the prompt and the LLM

In [4]:
import os
from openai import OpenAI

# Initialize the LLM
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Initalize the system prompt
messages = [
    {
        "role": "system",
        "content": "You are a Mastermind (a numbers game) playing agent.",
    },
]

Initialize the mastermind driver and get the first observation

In [5]:
driver = MasterMindDriver(game)
obs = driver.reset()
print(f"OBSERVATION: {obs.output}")

OBSERVATION: You are tasked to play the Mastermind game.
The host chooses a number and gives you the amount of digits. You have to guess the correct number as fast as you can.
The number can contain repetitions and any possible digit between: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.
At each round, you provide a number as a guess. At each step, the host provides you this information:
1. The number of correct digits in the wrong position.
2. The number of correct digits in the correct position.
The game ends when the host outputs 'You Won!'
Carefully choose your strategy. Avoid brute force.
The guess must be in the following format:
Guess: <number>
Start guessing the 4 digits number.


Start the LLM loop solving the task

In [7]:
actions, progress, repetitions = [], [], []
step_cnt = 0
while not obs.success and obs.can_proceed and step_cnt < 5:
    # Provide the observation to the LLM
    messages.append({"role": "user", "content": obs.output})

    # Get the LLM output
    agent_choice = (
        client.chat.completions.create(model="gpt-4", messages=messages)
        .choices[0]
        .message.content
    )

    # Provide the choice to the LLM as history
    messages.append({"role": "assistant", "content": agent_choice})
    print(agent_choice)

    # Run a mastermind round and get the observation
    obs = driver.step_raw(agent_choice)
    print(obs.output)

    step_cnt += 1

Guess: 3041
Wrong! Your guess has 0 correct digit in the correct position and 2 correct digits in the wrong positions. Keep guessing.
Guess: 1403
Wrong! Your guess has 1 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.
Guess: 1430
Wrong! Your guess has 2 correct digits in the correct positions and 0 correct digit in the wrong position. Keep guessing.
Guess: 1432
Wrong! Your guess has 2 correct digits in the correct positions and 0 correct digit in the wrong position. Keep guessing.
Guess: 1435
Wrong! Your guess has 2 correct digits in the correct positions and 0 correct digit in the wrong position. Keep guessing.


In [6]:
driver.metrics.export(
    repetition_function_kwargs={"theta_a": 1, "num_execution_steps": 5}
)

{'goal': '9439',
 'success': False,
 'actions': [{'value': '1234'},
  {'value': '1253'},
  {'value': '3410'},
  {'value': '4310'},
  {'value': '341'}],
 'states': [{'value': '1234'},
  {'value': '1253'},
  {'value': '3410'},
  {'value': '4310'},
  {'value': '341'}],
 'observations': [{'output': 'Wrong! Your guess has 1 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 1 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 0 correct digit in the correct position and 2 correct digits in the wrong positions. Keep guessing.',
   'success': False,
   'can_proc